In [126]:
# importa módulos
import pandas as pd
from unidecode import unidecode

In [127]:
# lê arquivos
escolas = pd.read_csv("escolas.csv")
material_didatico = pd.read_csv("material_didatico.csv")
subprefeituras = pd.read_csv("subprefeituras.csv")

In [128]:
material_didatico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          151 non-null    int64 
 1   Quantidade  147 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


a.1) converter tipo coluna de objeto para inteiro
    - substituir NaN e None por 0
    - substituir '' e ' ' por 0
    - substituir 'NaN' por 0
    - converter valor numérico do tipo objeto para int (Ex.: '356' para 356)

## Tratamento da tabela material_didatico

In [129]:
material_didatico_copy_1 = material_didatico.copy()

In [130]:
# normalizar nomes das colunas
cols = material_didatico_copy_1.columns
for col in cols:
    material_didatico_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [131]:
# remove valores nulos
material_didatico_copy_2 = material_didatico_copy_1.copy()
material_didatico_copy_2['quantidade'].fillna('0', inplace=True)

In [132]:
# converter todos os valores da coluna para int
material_didatico_copy_2['quantidade'] = material_didatico_copy_2['quantidade'].apply(lambda x: int(x) if x.isdigit() else 0)

In [133]:
# # Modo 2
# import re

# def normaliza_quantidade(valor) -> int:
#     if valor:
#         normaliza_valor = re.sub(r"[^\d: ]", "", valor.strip())
#         if normaliza_valor:
#             return int(normaliza_valor)
#         else:
#             return 0
#     else:
#         return 0
    
# material_didatico['quantidade'] = material_didatico['quantidade'].apply(lambda x: normaliza_quantidade(x))

In [134]:
material_didatico_copy_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          151 non-null    int64
 1   quantidade  151 non-null    int64
dtypes: int64(2)
memory usage: 2.5 KB


In [178]:
material_didatico_copy_2

,id,quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67
...,...,...
146,474,320
147,301,335
148,215,0
149,606,521


## Tratamento da tabela subprefeitura

In [135]:
subprefeituras_copy_1 = subprefeituras.copy()

In [136]:
# normalizar nomes das colunas
cols = subprefeituras_copy_1.columns
for col in cols:
    subprefeituras_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [137]:
# converter letras em minúsculos e remover acentos
subprefeituras_copy_2 = subprefeituras_copy_1.copy()
subprefeituras_copy_2 = subprefeituras_copy_2.applymap(lambda x: unidecode(x.upper()))

In [138]:
subprefeituras_copy_2

,nome,subprefeitura
0,SANTO CRISTO,CENTRO
1,GAMBOA,CENTRO
2,SAUDE,CENTRO
3,CAJU,CENTRO
4,CENTRO,CENTRO
...,...,...
159,SANTA CRUZ,ZONA OESTE
160,GUARATIBA,ZONA OESTE
161,BARRA DE GUARATIBA,ZONA OESTE
162,PEDRA DE GUARATIBA,ZONA OESTE


## Tratamento da tabela escolas

In [139]:
escolas_copy_1 = escolas.copy()

In [140]:
# normalizar nomes das colunas
cols = escolas_copy_1.columns
for col in cols:
    escolas_copy_1.rename(columns={col: unidecode(col.lower().strip())}, inplace=True)

In [141]:
# normaliza coluna id (3 caracteres. Ex: '024')
# Modo 1
escolas_copy_2 = escolas_copy_1.copy()
escolas_copy_2['id'] = escolas_copy_2['id'].apply(lambda x: str(x).zfill(3))

In [142]:
# # Modo 2
# escolas_copy_2_modo2 = escolas_copy_1.copy()
# escolas_copy_2_modo2['id'] = escolas_copy_2_modo2['id'].astype(str).str.zfill(3)

In [143]:
# Modo 1
# normalizar coluna lat com 5 casas decimais
escolas_copy_3 = escolas_copy_2.copy()
for col in ['lat', 'lon']:
    escolas_copy_3[col] = escolas_copy_3[col].str.replace(",", ".").astype(float).apply(lambda x: round(x, 5))

In [144]:
# # Modo 2
# # normalizar coluna lat com 5 casas decimais
# escolas_copy_3_modo2 = escolas_copy_2.copy()
# for col in ['lat', 'lon']:
#     escolas_copy_3_modo2[col] = escolas_copy_3_modo2[col].str.replace(",", ".").astype(float).round(5)

In [145]:
escolas_copy_4 = escolas_copy_3.copy()

Index(['id', 'escolas_postos', 'bairro', 'endereco', 'lat', 'lon'], dtype='object')

In [148]:
# normalizar colunas (remove acentos e string em maiúsculo)
escolas_copy_4 = escolas_copy_3.copy()
escolas_copy_4[['escolas_postos', 'bairro', 'endereco']] = escolas_copy_4[['escolas_postos', 'bairro', 'endereco']].applymap(lambda x: unidecode(x.upper()))

#### Substituir abreviações de endereço

In [175]:
escolas_copy_5 = escolas_copy_4.copy()

In [176]:
# verificar possíveis abreviações
abreviacoes = set()

for i, linha_escola in escolas_copy_5.iterrows():
    lista_palavra_endereco = linha_escola['endereco'].split()
    for palavra in lista_palavra_endereco:
        if len(palavra) <= 3 and not palavra.isdigit():
            abreviacoes.update([palavra])

abreviacoes

{'ANA',
 'AV',
 'AV.',
 'DA',
 'DAS',
 'DE',
 'DO',
 'DOM',
 'DOS',
 'E',
 'JR',
 'KM',
 'PAU',
 'R.',
 'RIO',
 'RUA',
 'S/N',
 'SD.',
 'UBA'}

In [177]:
# verificar possíveis abreviações para logradouros
filtro_abreviacoes = [abreviacao for abreviacao in abreviacoes if abreviacao.startswith('R') or abreviacao.startswith('AV')]
filtro_abreviacoes

['RIO', 'RUA', 'R.', 'AV', 'AV.']

In [170]:
# normalizar coluna endereco (substituir abreviações de logradouro)
escolas_copy_5 = escolas_copy_4.copy()
escolas_copy_5['endereco'] = escolas_copy_5['endereco'].str.replace('R. ', 'RUA ', regex=True).replace('AV. ', 'AVENIDA ', regex=True).replace('AV ', 'AVENIDA ', regex=True)
escolas_copy_5

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56",-22.90929,-43.18358
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64355
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No",-22.95316,-43.57741
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,-23.00908,-43.53758
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,-22.99412,-43.59368


In [174]:
# valida normalização de abreviações do campo endereco
for i, linha_escola_2 in escolas_copy_5.iterrows():
    if linha_escola_2['endereco'].startswith('R. ')  or \
       linha_escola_2['endereco'].startswith('AV. ') or \
       linha_escola_2['endereco'].startswith('AV '):
        print(linha_escola_2)

## Faltam os seguintes tratamentos:

- adicionar vírgula para o número do endereço
- separar logradouro e número do endereço